In [14]:
import pandas as pd
df=pd.read_csv(
    r"https://raw.githubusercontent.com/cior666/Machine-Learning-III-Entrega/refs/heads/main/Data%20preprocessing/spotify_reviews_preprocessed.csv"
)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54708 entries, 0 to 54707
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   processed_text  54708 non-null  object
 1   sentiment       54708 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 854.9+ KB


In [16]:
#vamos a proceder a hacer el split train y test
from sklearn.model_selection import train_test_split
X = df['processed_text']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [17]:
#Armamos el BOW
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer(
    min_df=2,        #aca eliminamos palabras poco comunes
    max_df=0.95,     #aca eliminamos palabras muy comunes
    ngram_range=(1,2)
)
X_train_bow = bow.fit_transform(X_train)
X_test_bow  = bow.transform(X_test)
X_train_bow.shape, X_test_bow.shape

((43766, 58023), (10942, 58023))

In [18]:
#hacemos el TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    min_df=2,
    max_df=0.95,
    ngram_range=(1,2)
)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)
X_train_tfidf.shape, X_test_tfidf.shape


((43766, 58023), (10942, 58023))

In [19]:
#procedemos a calcular el sparsity, esto nos indica el porcentaje de ceros que tiene la matriz de features.
import numpy as np
def sparsity(X):
    return 1.0 - (X.nnz / (X.shape[0] * X.shape[1]))

print("Sparsity BoW:", sparsity(X_train_bow))
print("Sparsity TF-IDF:", sparsity(X_train_tfidf))




Sparsity BoW: 0.9996237126983988
Sparsity TF-IDF: 0.9996237126983988


In [20]:
#Al obtener valores altos de sparsity, podemos concluir que cada reseña usa muy pocas palabras del vocabulario total
#por lo que la matriz es de alta dimensionalidad y asi mismo muy dispersa.
#Además en base a estos valores del sparsity nos vamos a inclinar por usar un modelo lineal que son los que mejor trabajan con estos casos.


In [21]:
#Planteamos el ver las palabras mas frecuentes para el BoW
import numpy as np
word_counts = X_train_bow.sum(axis=0)
words_freq = pd.DataFrame({
    'word': bow.get_feature_names_out(),
    'count': np.asarray(word_counts).flatten()
}).sort_values(by='count', ascending=False)
words_freq.head(10)


,word,count
3710,app,26470
45298,song,22264
30298,music,18604
46787,spotify,13509
35686,play,13203
32016,not,12513
13333,do,10375
19078,good,9261
25943,listen,8661
13458,do not,8455
